In [396]:
from pathlib import Path
from IPython.display import display
from functools import partial
import pickle
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import joblib

from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, recall_score, accuracy_score, precision_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline

import optuna

import interpret
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression
interpret.set_visualize_provider(interpret.provider.InlineProvider())

from nam.data import NAMDataset
from nam.config import defaults
from nam.data import FoldedDataset
from nam.models import NAM
from nam.models import get_num_units
from nam.trainer import LitNAM
from nam.types import Config
from nam.utils import parse_args
from nam.utils import plot_mean_feature_importance
from nam.utils import plot_nams
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

#### Notes
- Kroll: income proejctions from Kroll (Kroll Net Cash Flow NCF) kroll data spalte DSCR_NCF, YearBuilt, orig (originator?), PropertyType, Deal (ID spalte), PropertyType (muss geschaut werden was abkürzungen bedeuten)
- Green-Street: initial price paid for CMBS tranches
- Green-Street: Risk Retention?
- ExecuComp: Executive Compensation
- SEC filings: Risk management practices
- Boomberg: Information on fixed income underwrtigin 'league tables'

In [ ]:
# all_data = pd.read_excel('All_Data.xlsx', sheet_name=None)
# all_data = pd.read_excel('All_Data.xlsx', sheet_name='CMBSMainDataFile.dta')
with open('all_data.pickle', 'rb') as file:
    all_data = pickle.load(file)
all_data

#Non-Perorming: A dummy variable equal to one if a loan becomes non performing at any point from April 2020 to April 2021. A loan is non-perfomring if it has missed a payment, is in the process of foreclosure, is real estate owned, or has been placed into special servicing.
cmbs_main = all_data['CMBSMainDataFile.dta']
# cmbs_main = all_data
cmbs_main[
    [
        'non_perf', 
        #'non_perf2',
    ]
].hist()


# Distress: A variable equal to one if a loan appears on a servicer's watchlist or becomes non-perfomring from April 2020 to April 2021. From variables AprilWatch, MayWatch, JuneWatch, JulyWatch, Augwatch, Sepwatch, Oct20Watch, Nov20Watch, Dec20Watch, Jan21Watch, Feb21Watch, Mar21Watch, Apr21Watch
#cmbs_main = all_data['CMBSMainDataFile.dta']
distress = cmbs_main[['Distress']]
cmbs_main[
    [
        'Distress', 
    ]
].hist()

list(all_data.keys())

deal_characteristics.dtypes

len(cmbs_main)

# EDA
additional_columns = [
    'Originator',
    'ApprValUSD',
    'CapRate',
    'log_bal',
    'LTV',
    'DSCR',
    'Size',
    'RateType',
    'buildingage',
    'NOI',
    'Occupancy',
    'CutoffLTV',
    'CutoffCpn',
    'CutoffOcc',
    'CutoffDSCR',
    'CutoffNOIUSD',
    'CutoffNCFUSD', 
    'fixed',
] 

# age
# Rate type
# cmbs_main.columns.tolist()

data = cmbs_main[
    [
        'Deal', # ID Column
        'OVER_w',
        'past_over',
        'high_overstatement2', # is 100% dependent on Over_w, if we predict this we get 100% accuracy
        'Distress',
        'non_perf'
    ] + additional_columns
]
data = data.replace(' ', pd.NA)

# Encode categorical data
data['Originator'] = LabelEncoder().fit_transform(data['Originator'])
data['RateType'] = LabelEncoder().fit_transform(data['RateType'])

clean_data = data[
    data.notna().all(axis=1)
]
clean_data

clean_data = clean_data.astype({'Occupancy': float})

In [422]:
orig_characteristics = all_data['OrigCharacteristics.dta']
orig_characteristics_columns = [
    #'Deal',
    'type',
    'CutoffLTV',
    'CutoffDSCR',
    'CutoffCpn',
    'log_bal',
    'fixed',
    'buildingage',
    'CutoffOcc',
    'year_priced',
    'quarter_type',
    'AmortType',
    # 'MSA',
    'qy',
    'Size',

    'OVER_w',
    'past_over',
    'high_overstatement2', # is 100% dependent on Over_w, if we predict this we get 100% accuracy
    'Distress',
    #'non_perf'
]
orig_data = orig_characteristics[orig_characteristics_columns]
# orig_characteristics['Originator'] = LabelEncoder().fit_transform(data['Originator'])
# orig_characteristics['RateType'] = LabelEncoder().fit_transform(data['RateType'])

In [ ]:
target_col = 'Distress'

In [423]:
orig_data_with_dummies = pd.get_dummies(
    orig_data,
    columns=[
        'AmortType',
        # 'MSA',
        'type'
    ]
)
clean_data = orig_data_with_dummies[
    orig_data_with_dummies.notna().all(axis=1)
]
clean_data

,CutoffLTV,CutoffDSCR,CutoffCpn,log_bal,fixed,buildingage,CutoffOcc,year_priced,quarter_type,qy,...,AmortType_Fully Amort,AmortType_IO,AmortType_Partial IO,type_AGENCY-DUS,type_Agency,type_Conduit,type_Large Loan,type_Other,type_SASB,type_Small Bal
1,0.664,1.340,4.990,16.518549,0.0,2.944439,0.954,2015.0,192.0,32018.0,...,False,True,False,False,True,False,False,False,False,False
4,0.750,1.316,4.140,14.440914,0.0,2.890372,0.949,2013.0,114.0,22019.0,...,False,False,True,False,True,False,False,False,False,False
6,0.587,1.266,4.410,16.871868,0.0,3.806663,1.000,2019.0,47.0,32015.0,...,False,True,False,False,True,False,False,False,False,False
7,0.659,1.375,4.010,16.133181,1.0,0.000000,0.962,2013.0,60.0,42014.0,...,False,False,False,False,True,False,False,False,False,False
8,0.742,1.380,3.790,17.504391,1.0,4.753590,0.924,2018.0,127.0,32015.0,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40038,0.700,1.580,3.830,13.840203,1.0,2.890372,0.943,2020.0,198.0,42012.0,...,False,False,True,False,False,True,False,False,False,False
40040,0.743,1.570,5.910,15.800641,1.0,2.944439,0.983,2018.0,265.0,22019.0,...,False,True,False,False,False,True,False,False,False,False
40041,0.642,1.318,3.948,17.448877,1.0,1.609438,0.984,2017.0,253.0,42007.0,...,False,False,False,False,False,True,False,False,False,False
40042,0.732,1.900,4.838,16.491037,1.0,0.693147,0.985,2015.0,41.0,12015.0,...,False,False,True,False,True,False,False,False,False,False


In [424]:
dummy_cols = [col for col, dtype in clean_data.dtypes.items() if dtype == bool]
for dummy_col in dummy_cols:
    clean_data[dummy_col] = clean_data[dummy_col].map({True: 1, False:0})

/var/folders/pr/tf6fbkqx6mv5m13cy7d765fw0000gn/T/ipykernel_8654/3247692996.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [425]:
# Percentage of clean data from whole dataset
len(clean_data) / len(orig_data_with_dummies)

0.6110375827194406

In [426]:
y = clean_data[target_col].astype('U32')
X = clean_data.drop(columns=target_col)

In [427]:
X

,CutoffLTV,CutoffDSCR,CutoffCpn,log_bal,fixed,buildingage,CutoffOcc,year_priced,quarter_type,qy,...,AmortType_Fully Amort,AmortType_IO,AmortType_Partial IO,type_AGENCY-DUS,type_Agency,type_Conduit,type_Large Loan,type_Other,type_SASB,type_Small Bal
1,0.664,1.340,4.990,16.518549,0.0,2.944439,0.954,2015.0,192.0,32018.0,...,0,1,0,0,1,0,0,0,0,0
4,0.750,1.316,4.140,14.440914,0.0,2.890372,0.949,2013.0,114.0,22019.0,...,0,0,1,0,1,0,0,0,0,0
6,0.587,1.266,4.410,16.871868,0.0,3.806663,1.000,2019.0,47.0,32015.0,...,0,1,0,0,1,0,0,0,0,0
7,0.659,1.375,4.010,16.133181,1.0,0.000000,0.962,2013.0,60.0,42014.0,...,0,0,0,0,1,0,0,0,0,0
8,0.742,1.380,3.790,17.504391,1.0,4.753590,0.924,2018.0,127.0,32015.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40038,0.700,1.580,3.830,13.840203,1.0,2.890372,0.943,2020.0,198.0,42012.0,...,0,0,1,0,0,1,0,0,0,0
40040,0.743,1.570,5.910,15.800641,1.0,2.944439,0.983,2018.0,265.0,22019.0,...,0,1,0,0,0,1,0,0,0,0
40041,0.642,1.318,3.948,17.448877,1.0,1.609438,0.984,2017.0,253.0,42007.0,...,0,0,0,0,0,1,0,0,0,0
40042,0.732,1.900,4.838,16.491037,1.0,0.693147,0.985,2015.0,41.0,12015.0,...,0,0,1,0,1,0,0,0,0,0


In [ ]:
# X, X_leave_out, y, y_leave_out = train_test_split(input_data, target, stratify=target, random_state=1, shuffle=True, test_size=0.2)

In [ ]:
# X_train_oversampled, y_train_oversampled = SMOTE().fit_resample(X_train, y_train)
# X_train_oversampled.hist()
# X_train_oversampled['CapRate'].plot()
# y_train_oversampled.hist()

In [428]:
sample_size = len(y)
sample_size

24469

## Training of models

### EBM (Explainable Boosting Machine)

In [ ]:
# ebm_model_undersample = make_pipeline(
#         RandomUnderSampler(random_state=42),
#         ExplainableBoostingClassifier(random_state=42)
# )
# ebm_model_cv = cross_validate(
#     ebm_model_undersample, X, y, scoring='roc_auc',
#     return_train_score=True, return_estimator=True,
#     n_jobs=-1 
# )
# print(
#     f"ROC AUC mean +/- std. dev.: "
#     f"{ebm_model_cv['test_score'].mean():.3f} +/- "
#     f"{ebm_model_cv['test_score'].std():.3f}"
# )

In [433]:
global_res_models = {}
# objective for optuna hyperparameter search
def objective(trial, clf_type, X, y):
    res = {
        'scores': [],
        'models': [],
    }
    if clf_type == 'EBM':
        learning_rate = trial.suggest_float('learning_rate', 0.0009, 0.0015, step=0.0004)
        smoothing_rounds = trial.suggest_trial('smoothing_rounds',low=0, high=4, step=2), # default is 0

        model_name = "EBM"
        clf = ExplainableBoostingClassifier(
            random_state=42,
            learning_rate=learning_rate,
            smoothing_rounds=smoothing_rounds
        )
    
    if clf_type == 'LogisticRegression':
        penalty = trial.suggest_categorical("penalty", ["elasticnet", "l2", 'l1'])
        C = trial.suggest_float("C", 0.001, 2, step=0.5)
        class_weight = trial.suggest_categorical('class_weight', ['balanced', None])
        solver = trial.suggest_categorical('solver', ['newton-cholesky', 'lbfgs'])

        model_name = "LogisticRegression"
        clf = LogisticRegression(
            random_state=42,
            penalty=penalty,
            C=C,
            class_weight=class_weight,
            solver=solver
        )

    # Use stratified K Fold instead of normal k fold to preserve class imbalance
    strat_kfold = StratifiedKFold(n_splits=2)
    # for train_idx, test_idx in tqdm(strat_kfold.split(X, y), total=strat_kfold.get_n_splits(), desc='K fold'):
    for fold, (train_idx, test_idx) in enumerate(strat_kfold.split(X, y)):
        print(f'Currently in fold: {fold}')
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]

        X_test = X.iloc[test_idx, :]
        y_test = y.iloc[test_idx]
        X_train_oversampled, y_train_oversampled = SMOTE().fit_resample(X_train, y_train)
        print('Oversampled data')
        print('Starting to fit data')
        start_time = time.perf_counter()
        clf.fit(X_train_oversampled, y_train_oversampled)
        total_time = time.perf_counter() - start_time
        print(f'Finished fitting data, took {total_time} sec.')

        auc_roc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        res['scores'].append(auc_roc)
        res['models'].append(clf)

    trial.set_user_attr(key='scores_and_models', value=res)
    global_res_models.update({model_name: res})
    return np.mean(auc_roc)

# current best 0.54
study_ebm = optuna.create_study(direction='maximize', study_name='EBM_study')
objective_edm = partial(objective, clf_type='EBM', X=X, y=y)
study_ebm.optimize(objective_edm, n_trials=1, show_progress_bar=True)
# no current best
study_LR = optuna.create_study(direction='maximize', study_name='LogisticRegression_study')
objective_LR = partial(objective, clf_type='LogisticRegression', X=X, y=y)
study_LR.optimize(objective_LR, n_trials=1, show_progress_bar=True)


[I 2023-09-05 18:46:23,936] A new study created in memory with name: EBM_study
  0%|          | 0/1 [00:00<?, ?it/s]/Users/janik/Documents/Master/KIT/Semester 4/Advanced Machine Learning Projekt/.venv/lib/python3.11/site-packages/optuna/distributions.py:685: UserWarning:

The distribution is specified by [0.0009, 0.0015] and step=0.0004, but the range is not divisible by `step`. It will be replaced by [0.0009, 0.0013].



Currently in fold: 0
Oversampled data
Starting to fit data


  0%|          | 0/1 [00:11<?, ?it/s]

[W 2023-09-05 18:46:35,698] Trial 0 failed with parameters: {'learning_rate': 0.0013} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/janik/Documents/Master/KIT/Semester 4/Advanced Machine Learning Projekt/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/pr/tf6fbkqx6mv5m13cy7d765fw0000gn/T/ipykernel_8654/4098240991.py", line 44, in objective
    clf.fit(X_train_oversampled, y_train_oversampled)
  File "/Users/janik/Documents/Master/KIT/Semester 4/Advanced Machine Learning Projekt/.venv/lib/python3.11/site-packages/interpret/glassbox/_ebm/_ebm.py", line 894, in fit
    results = provider.parallel(boost, parallel_args)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/janik/Documents/Master/KIT/Semester 4/Advanced Machine Learning Projekt/.venv/lib/python3.11/site-packages/interpret/provide

KeyboardInterrupt: 

In [ ]:
from interpret import show
from interpret import set_visualize_provider
from interpret.provider import InlineProvider

set_visualize_provider(InlineProvider())
show(ebm.explain_global())

In [ ]:
show(ebm.explain_global())

In [ ]:
confusion_matrix(y_true=target.values, y_pred=y_pred)

In [ ]:
cm = confusion_matrix(y_true=target.values, y_pred=y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
interpret.show(ROC(ebm_cv, X.columns).explain_perf(X, target))

In [ ]:
# class imbalance
n_positives = len(y_train[y_train == '1.0'])
n_negatives = len(y_train[y_train == '0.0'])
ratio = n_positives / (n_positives + n_negatives)
print(f'Ratio of samples is: {1-ratio}')


# Calculate Scores
acc_score = accuracy_score(target, y_pred)
auc_roc = roc_auc_score(target, ebm_cv.predict_proba(X)[:, 1])
print(f'Accuracy score: {acc_score}')
print(f'ROC AUC score: {auc_roc}')


In [ ]:
# ebm_model_smote = make_pipeline(
#         SMOTE(random_state=42),
#         ExplainableBoostingClassifier(random_state=42)
# )
# ebm_model_cv_smote = cross_validate(
#     ebm_model_smote, X, y, scoring='roc_auc',
#     return_train_score=True, return_estimator=True,
#     n_jobs=-1 
# )
# print(
#     f"ROC AUC mean +/- std. dev.: "
#     f"{ebm_model_cv_smote['test_score'].mean():.3f} +/- "
#     f"{ebm_model_cv_smote['test_score'].std():.3f}"
# )
# ebm_model_cv_smote
# ebm_model_cv
# ebm = ExplainableBoostingClassifier(random_state=42)
# # study = optuna.create_study(study_name='AML_EBM_study')
# param_distributions = {
#     'learning_rate': optuna.distributions.FloatDistribution(low=0.0009, high=0.0015, step=0.0004), # default is 0.001, best trial 0.0009
#     # 'greediness': optuna.distributions.FloatDistribution(low=0.0, high=0.01, step=0.001), # default is 0
#     # 'outer_bags': optuna.distributions.IntDistribution(low=5, high=10, step=1), # default is 8
#     # 'inner_bags': optuna.distributions.IntDistribution(low=0, high=3), # default is 0
#     # 'max_rounds': optuna.distributions.IntDistribution(low=4500, high=5500, step=500), # default is 5000
#     'smoothing_rounds': optuna.distributions.IntDistribution(low=0, high=4, step=2), # default is 0
#     # 'early_stopping_rounds': optuna.distributions.IntDistribution(low=40, high=60, step=15), # default is 50
# }
# ebm_cv = optuna.integration.OptunaSearchCV(
#     ebm_model_undersample,
#     param_distributions,
#     n_trials=5,
#     cv=4, # search with 3 fold CV
#     # study=study
# ) 

# ebm_cv.fit(X, y)
# joblib.dump(ebm_cv, 'models/ebm_optuna_learning_rate.joblib')
# y_pred = ebm_cv.predict(X)
# y_pred
# np.unique(y_pred)
# len(y_pred[y_pred == 0])/ len(y_pred[y_pred == 1])
# plt.hist(y_pred)
# ebm_model_smote = make_pipeline(
#         SMOTE(random_state=42),
#         ExplainableBoostingClassifier(random_state=42)
# )
# ebm_model_cv_smote = cross_validate(
#     ebm_model_smote, X, y, scoring='roc_auc',
#     return_train_score=True, return_estimator=True,
#     n_jobs=-1 
# )
# print(
#     f"ROC AUC mean +/- std. dev.: "
#     f"{ebm_model_cv_smote['test_score'].mean():.3f} +/- "
#     f"{ebm_model_cv_smote['test_score'].std():.3f}"
# )
# ebm_model_cv_smote
# ebm_model_cv
# ebm = ExplainableBoostingClassifier(random_state=42)
# # study = optuna.create_study(study_name='AML_EBM_study')
# param_distributions = {
#     'learning_rate': optuna.distributions.FloatDistribution(low=0.0009, high=0.0015, step=0.0004), # default is 0.001, best trial 0.0009
#     # 'greediness': optuna.distributions.FloatDistribution(low=0.0, high=0.01, step=0.001), # default is 0
#     # 'outer_bags': optuna.distributions.IntDistribution(low=5, high=10, step=1), # default is 8
#     # 'inner_bags': optuna.distributions.IntDistribution(low=0, high=3), # default is 0
#     # 'max_rounds': optuna.distributions.IntDistribution(low=4500, high=5500, step=500), # default is 5000
#     'smoothing_rounds': optuna.distributions.IntDistribution(low=0, high=4, step=2), # default is 0
#     # 'early_stopping_rounds': optuna.distributions.IntDistribution(low=40, high=60, step=15), # default is 50
# }
# ebm_cv = optuna.integration.OptunaSearchCV(
#     ebm_model_undersample,
#     param_distributions,
#     n_trials=5,
#     cv=4, # search with 3 fold CV
#     # study=study
# ) 

# ebm_cv.fit(X, y)
# joblib.dump(ebm_cv, 'models/ebm_optuna_learning_rate.joblib')
# y_pred = ebm_cv.predict(X)
# y_pred
# np.unique(y_pred)
# len(y_pred[y_pred == 0])/ len(y_pred[y_pred == 1])
# plt.hist(y_pred)

### Logisitic Regression

In [ ]:
# Logisitc Regression or XGBoost

### NAM

In [ ]:
x = np.random.normal(0,2, (500,4))
clean_data = pd.DataFrame(x, columns= ['a', 'b', 'c', 'd'])
target_col = 'a'
clean_data
config
# Neural Additive Model
config = defaults()
feature_cols = [col for col in clean_data.columns if col != target_col]
nam_dataset = NAMDataset(
    config,
    data_path=clean_data,
    features_columns=feature_cols,
    targets_column=target_col,
)
nam_model = NAM(
    config=config,
    name='Testing_NAM',
    num_inputs=len(nam_dataset[0][0]),
    num_units=get_num_units(config, nam_dataset.features)
)
nam_model
# NAM Training
data_loaders = nam_dataset.train_dataloaders()
for fold, (train_loader, val_loader) in enumerate(data_loaders):
     tb_logger = TensorBoardLogger(
          save_dir=config.logdir,
          name=f'{nam_model.name}',
          version=f'fold_{fold + 1}')

     checkpoint_callback = ModelCheckpoint(
          filename=tb_logger.log_dir + "/{epoch:02d}-{val_loss:.4f}",
          monitor='val_loss',
          save_top_k=config.save_top_k,
          mode='min'
     )
     litmodel = LitNAM(config, nam_model)
     pl.Trainer()
     trainer = pl.Trainer(
          logger=tb_logger,
          max_epochs=config.num_epochs,
          callbacks=checkpoint_callback,
     )
     trainer.fit(
          litmodel,
          train_dataloaders=train_loader,
          val_dataloaders=val_loader)
trainer.logger.experiment.summary

### NAM Testing and Evaluation

In [ ]:
trainer.test(litmodel, dataloaders=nam_dataset.test_dataloaders())
fig = plot_mean_feature_importance(litmodel.model, nam_dataset)
fig = plot_nams(litmodel.model, nam_dataset, num_cols= 3)

# Evaluation

## Anhang

from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
param_test = {'learning_rate': [0.001,0.005,0.01,0.03],
              'interactions': [5,10,15],
              'max_interaction_bins': [10,15,20],
              'max_rounds': [5000,10000,15000,20000],
              'min_samples_leaf': [2,3,5],
              'max_leaves': [3,5,10]}
n_HP_points_to_test=10
LGBM_clf = LGBMClassifier(random_state=314, n_jobs=-1)
LGBM_gs = RandomizedSearchCV(
    estimator=LGBM_clf,
    param_distributions=param_test,
    n_iter=n_HP_points_to_test,
    scoring="roc_auc",
    cv=3,
    refit=True,
    random_state=314,
    verbose=False,
)
LGBM_gs.fit(X_train, y_train)

for df_name, df in all_data.items():
    display(df_name)
    display(df.columns.to_list())
    display('------------')

def get_scores(clf, X_test, y_true, print_results=False):
    scores = {}
    y_pred = clf.predict(X_test)
    scores['accuracy'] = accuracy_score(y_true=y_true, y_pred=y_pred)
    scores['recall'] = recall_score(y_true=y_true, y_pred=y_pred)
    scores['roc_auc'] = roc_auc_score(y_true=y_true, y_score=clf.predict_proba(X_test))

    if print_results:
        for score_name, score in scores.items():
            print(f'{score_name}: {score}')

    return scores